In [3]:
import pandas as pd
import regex as re
import glob,os
from tika import parser
from Bio import Entrez

In [15]:
Entrez.email = 'User email address'

In [3]:
handle = Entrez.esearch(db = 'pubmed', term = "Mass spectrometry AND lipid AND brain")
record = Entrez.read(handle)
count = int(record['Count'])
handle = Entrez.esearch(db = 'pubmed', term = "Mass spectrometry AND lipid AND brain", retmax=count)
record = Entrez.read(handle)
handle.close() #Search on Pubmed

In [4]:
with open('file_1.txt', 'w') as file:
    for i in record['IdList']:
        print(i + ',', end="", file=file) #Save the PMIDs in a text file

Once the file is downloaded containing the PMIDs, use the fetch_pdfs.py script to download the journal articles. 

The script can be found in the following link,

https://github.com/billgreenwald/Pubmed-Batch-Download

An example of usage, 

Python3 fetch_pdfs.py -pmids 1234,6789 

Once the files are downloaded, continue with the next step 

In [4]:
import glob, os

path = r'D:\fetched_pdfs' #the path where the pdf files are
    
os.chdir(path)
pdfs = []
for file in glob.glob("*.pdf"):
    pdfs.append(file) #get the names of all pdf files 

In [5]:
def isFullPdf(f): 
    
    '''Check for broken PDF files'''
    
    end_content = ''
    start_content = ''
    size = os.path.getsize(f)
    if size < 1024: return False 
    with open(f, 'rb') as fin: 
        #start content 
        fin.seek(0, 0)
        start_content = fin.read(1024)
        start_content = start_content.decode("ascii", 'ignore' )
        fin.seek(-1024, 2)
        end_content = fin.read()
        end_content = end_content.decode("ascii", 'ignore' )
    start_flag = False
    #%PDF
    if start_content.count('%PDF') > 0:
        start_flag = True
    
        
    if end_content.count('%%EOF') and start_flag > 0:
        return True
    eof = bytes([0])
    eof = eof.decode("ascii")
    if end_content.endswith(eof) and start_flag:
        return True
    return False

In [6]:
for file in pdfs:
    if os.path.getsize(file) < 12 * 1024 or isFullPdf(file)!= True: #remove broken files
        os.remove(file)

In [7]:
g = list()

x = 0

for i in pdfs:
    try:
        g.append(parser.from_file(i)) #read/parse all the files 
        g.append(i)
    except:
        pass

In [11]:
annots = pd.read_excel('annots_neg.xlsx') #read in a .xlsx file containing one column of putative annotations (one lipid per row)

In [14]:
annotations = set()

for x in (annots['annotations_neg']): #annotation dataframe[name of the column]
    for i in range(0,len(g),2):
        temp_1 = re.findall(x, g[i]['content'])  
        if len(temp_1) != 0:
            annotations.add(temp_1[0]) #Find if annotation is mentioned in the literature, and if so, append
            
print(annotations)

{'PS 40:6', 'PE 40:5', 'MGDG 34:1', 'PE 40:6', 'PS 38:4'}
